# Python + MongoDB

## Connect to MongoDB
To test if a connection to MongoDB can be established.

In [ ]:
import pymongo

try:
    # establish a mongoDB connection
    myclient = pymongo.MongoClient("localhost", 27017)
    connected = True
    
    # get any existing databases
    dblist = myclient.list_database_names()
    print("Existing Databases:")
    for db in dblist:
        print(db)
except pymongo.errors as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    connected = False

finally:
    if connected:
        myclient.close()

## Context Manager
`context managers` allow you to allocate and release resources precisely when you want to. `with` statement is widely used as exmple of `context manager`. <br>
We will be using the MongoDB <code>context manager</code> from this point onwards.<br>
*Note*: Pass a different <code>host</code> and <code>port</code> values where needed.

In [9]:
# implement a context manager as a class
from pymongo import MongoClient

class MongoDBClient(object):
    
    def __init__(self, host='localhost', port=27017): #class constructor with default arguments
        # MongoDB client
        try:
            self.client = None
            self.client = MongoClient(host, port)
            self.connected = True
        except:
            print("Error connecting to MongoDB Platform")
            self.connected = False
            
    # These magic methods (__enter__, __exit__) allows you to 
    # implement objects which can be used easily with the 'with' statement.
    
    # return members 
    def __enter__(self):
        print("Connecting to MongoDB...")
        return self.client
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("Exiting MongoDB...")
        if self.connected:
            self.client.close()
        
    print("Created class MongoDBClient successfully...")

Created class MongoDBClient successfully...


The code in the cell below is to display the *connection status of the `mongo client` object. <br>
It uses `with` statement. Let's talk about what's happening when executing the statement: <br>
- The `with` statement stores the `__exit__` method of the `MongoDBClient` class. <br>
- It calls the `__enter__` method of the `MongoDBClient` class. <br>
- The `__enter__` method creat an object and returns the memeber `client`. <br>
- The returned `client` is passed to `mClient`. <br>
- We print the *connection status of `mClient`. <br>
- The `with` statement calls the stored `__exit__` method. <br>
- The `__exit__` method closes the `client`.

In [11]:
# to view the connection status of mongo client object
with MongoDBClient() as mClient:
    print(mClient)

Connecting to MongoDB...
MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Exiting MongoDB...


## Creation of a database

Create a database called `mydatabase`, and 2 collections called `student` and `faculty`. Insert some data in these collections

In [13]:
student_list = [
    {"name": "Amy", "MatrNum": "0000154", "course": "BSc Mechanical Engineering"},
    {"name": "Jon", "MatrNum": "0000158", "course": "BSc Electronics Engineering"},
    {"name": "Michael", "MatrNum": "0000482", "course": "BSc Mechanical Engineering"},
    {"name": "Sandra", "MatrNum": "0000962", "course": "BSc Mechanical Engineering"},
    {"name": "Ben", "MatrNum": "0001058", "course": "BSc Systems Engineering"},
    {"name": "Tom", "MatrNum": "0000845", "course": "BSc Mechanical Engineering"},
    {"name": "Harry", "MatrNum": "0005436", "course": "BSc Life Sciences"},
    {"name": "Archie", "MatrNum": "0000968", "course": "BSc Mechanical Engineering"},
    {"name": "Philip", "MatrNum": "0000824", "course": "BSc Nursing"},
    {"name": "Stacy", "MatrNum": "0000156", "course": "BSc Early Childhood Education"},
    {"name": "Betty", "MatrNum": "0000578", "course": "BSc Mechanical Engineering"},
    {"name": "Richard", "MatrNum": "0002548", "course": "BSc Veterinary Science"}
]

faculty_list = [
    {"faculty": "Engineering"},
    {"faculty": "Life Sciences"},
    {"faculty": "Veterinary Medicine"}
]

print("Created 'student' and 'faculty' collections")

Created 'student' and 'faculty' collections


In [14]:
with MongoDBClient() as mClient:
    # create a database
    db = mClient["mydatabase"]
    # create a collection in the database
    db_col_student = db["student"]
    db_col_faculty = db["faculty"]
    
    # insert the records into the collection
    db_col_student.insert_many(student_list)
    db_col_faculty.insert_many(faculty_list)
    
    print("Insertion into collections done")

Connecting to MongoDB...
Insertion into collections done
Exiting MongoDB...


## Read data from the Collection

Difference between `find_one` and `find` functions

In [15]:
with MongoDBClient() as mClient:
    # accessing a database collection
    db_col_student = mClient.mydatabase.student
    
    print("First document of the Student Collection:\n", db_col_student.find_one())
    print("\nvs\n")
    print("All documents of Student Collection:\n")
    for i in db_col_student.find():
        print(i)

Connecting to MongoDB...
First document of the Student Collection:
 {'_id': ObjectId('624fa50308e830c41d77d0ee'), 'name': 'Amy', 'MatrNum': '0000154', 'course': 'BSc Mechanical Engineering'}

vs

All documents of Student Collection:

{'_id': ObjectId('624fa50308e830c41d77d0ee'), 'name': 'Amy', 'MatrNum': '0000154', 'course': 'BSc Mechanical Engineering'}
{'_id': ObjectId('624fa50308e830c41d77d0ef'), 'name': 'Jon', 'MatrNum': '0000158', 'course': 'BSc Electronics Engineering'}
{'_id': ObjectId('624fa50308e830c41d77d0f0'), 'name': 'Michael', 'MatrNum': '0000482', 'course': 'BSc Mechanical Engineering'}
{'_id': ObjectId('624fa50308e830c41d77d0f1'), 'name': 'Sandra', 'MatrNum': '0000962', 'course': 'BSc Mechanical Engineering'}
{'_id': ObjectId('624fa50308e830c41d77d0f2'), 'name': 'Ben', 'MatrNum': '0001058', 'course': 'BSc Systems Engineering'}
{'_id': ObjectId('624fa50308e830c41d77d0f3'), 'name': 'Tom', 'MatrNum': '0000845', 'course': 'BSc Mechanical Engineering'}
{'_id': ObjectId('624fa

Returning only the documents with 'BSc Mechanical Engineering' in the `course` field and do not show the `_id` key, value field.

In [16]:
with MongoDBClient() as mClient:
    # accessing a database collection
    db_col_student = mClient.mydatabase.student
    
    # 1st dict is to find documents the particular field value
    # 2nd dict tells which fields should be returned in the result
        # by default all fields are returned (1), a 0 means that field will 
        # not be returned
    docs = db_col_student.find({"course": "BSc Mechanical Engineering"}, 
                               {"_id": 0}, sort=[("name",1)])
    for i in docs:
        print(i)

Connecting to MongoDB...
{'name': 'Amy', 'MatrNum': '0000154', 'course': 'BSc Mechanical Engineering'}
{'name': 'Archie', 'MatrNum': '0000968', 'course': 'BSc Mechanical Engineering'}
{'name': 'Betty', 'MatrNum': '0000578', 'course': 'BSc Mechanical Engineering'}
{'name': 'Michael', 'MatrNum': '0000482', 'course': 'BSc Mechanical Engineering'}
{'name': 'Sandra', 'MatrNum': '0000962', 'course': 'BSc Mechanical Engineering'}
{'name': 'Tom', 'MatrNum': '0000845', 'course': 'BSc Mechanical Engineering'}
Exiting MongoDB...


Another way the `sort` function can be called is using the `cursor` object returned by the `find` function.

In [17]:
with MongoDBClient() as mClient:
    # accessing a database collection
    db_col_student = mClient.mydatabase.student
    
    docs = db_col_student.find({"course": "BSc Mechanical Engineering"},
                               {"_id": 0}).sort("name", 1)
    for i in docs:
        print(i)

Connecting to MongoDB...
{'name': 'Amy', 'MatrNum': '0000154', 'course': 'BSc Mechanical Engineering'}
{'name': 'Archie', 'MatrNum': '0000968', 'course': 'BSc Mechanical Engineering'}
{'name': 'Betty', 'MatrNum': '0000578', 'course': 'BSc Mechanical Engineering'}
{'name': 'Michael', 'MatrNum': '0000482', 'course': 'BSc Mechanical Engineering'}
{'name': 'Sandra', 'MatrNum': '0000962', 'course': 'BSc Mechanical Engineering'}
{'name': 'Tom', 'MatrNum': '0000845', 'course': 'BSc Mechanical Engineering'}
Exiting MongoDB...


Count the number of returned documents

In [18]:
with MongoDBClient() as mClient:
    # accessing a database collection
    db_col_student = mClient.mydatabase.student
    
    print(db_col_student.count_documents({"course": "BSc Mechanical Engineering"}))

Connecting to MongoDB...
6
Exiting MongoDB...


## Update data in the Collection
<br>
Update a single document

In [19]:
with MongoDBClient() as mClient:
    # accessing a database collection
    db_col_student = mClient.mydatabase.student
    
    print("Before update:")
    doc = db_col_student.find_one({"name": "Archie"}, {"_id": 0})
    print(doc)
    
    # update the student's course to 'Life Sciences'
    query = { "name": "Archie" }
    new_values = { "$set": { "course": "BSc Life Sciences" } }

    db_col_student.update_one(query, new_values)
    
    print("\nAfter update:")
    doc = db_col_student.find_one({"name": "Archie"}, {"_id": 0})
    print(doc)

Connecting to MongoDB...
Before update:
{'name': 'Archie', 'MatrNum': '0000968', 'course': 'BSc Mechanical Engineering'}

After update:
{'name': 'Archie', 'MatrNum': '0000968', 'course': 'BSc Life Sciences'}
Exiting MongoDB...


Updating multiple documents

In [20]:
with MongoDBClient() as mClient:
    # accessing a database collection
    db_col_student = mClient.mydatabase.student
    
    print("Before update:")
    docs = db_col_student.find({"name": { "$regex": "^S" }}, {"_id": 0})
    for i in docs:
        print(i)
    
    # update the course of students with the names starting with a
    # capital 'S' to 'BSc Veterinary Science'
    query = { "name": { "$regex": "^S" } }
    new_values = { "$set": { "course": "BSc Veterinary Science" } }

    db_col_student.update_many(query, new_values)
    
    print("\nAfter update:")
    docs = db_col_student.find({"name": { "$regex": "^S" }}, {"_id": 0})
    for i in docs:
        print(i)

Connecting to MongoDB...
Before update:
{'name': 'Sandra', 'MatrNum': '0000962', 'course': 'BSc Mechanical Engineering'}
{'name': 'Stacy', 'MatrNum': '0000156', 'course': 'BSc Early Childhood Education'}

After update:
{'name': 'Sandra', 'MatrNum': '0000962', 'course': 'BSc Veterinary Science'}
{'name': 'Stacy', 'MatrNum': '0000156', 'course': 'BSc Veterinary Science'}
Exiting MongoDB...


## Delete data from a Collection
<br>
Delete a single document

In [21]:
with MongoDBClient() as mClient:
    # accessing a database collection
    db_col_student = mClient.mydatabase.student
    
    print("Before delete:")
    docs = db_col_student.find({}, {"_id": 0})
    for i in docs:
        print(i)
    
    result = db_col_student.delete_one({"name": "Jon"})
    
    print(f"\n\n{result.deleted_count} document deleted\n")
    print("After delete:")
    docs = db_col_student.find({}, {"_id": 0})
    for i in docs:
        print(i)

Connecting to MongoDB...
Before delete:
{'name': 'Amy', 'MatrNum': '0000154', 'course': 'BSc Mechanical Engineering'}
{'name': 'Jon', 'MatrNum': '0000158', 'course': 'BSc Electronics Engineering'}
{'name': 'Michael', 'MatrNum': '0000482', 'course': 'BSc Mechanical Engineering'}
{'name': 'Sandra', 'MatrNum': '0000962', 'course': 'BSc Veterinary Science'}
{'name': 'Ben', 'MatrNum': '0001058', 'course': 'BSc Systems Engineering'}
{'name': 'Tom', 'MatrNum': '0000845', 'course': 'BSc Mechanical Engineering'}
{'name': 'Harry', 'MatrNum': '0005436', 'course': 'BSc Life Sciences'}
{'name': 'Archie', 'MatrNum': '0000968', 'course': 'BSc Life Sciences'}
{'name': 'Philip', 'MatrNum': '0000824', 'course': 'BSc Nursing'}
{'name': 'Stacy', 'MatrNum': '0000156', 'course': 'BSc Veterinary Science'}
{'name': 'Betty', 'MatrNum': '0000578', 'course': 'BSc Mechanical Engineering'}
{'name': 'Richard', 'MatrNum': '0002548', 'course': 'BSc Veterinary Science'}


1 document deleted

After delete:
{'name': 'Am

Delete multiple documents

In [22]:
with MongoDBClient() as mClient:
    # accessing a database collection
    db_col_student = mClient.mydatabase.student
    
    print("Before delete:")
    docs = db_col_student.find({}, {"_id": 0})
    for i in docs:
        print(i)
    
    result = db_col_student.delete_many({"course": "BSc Veterinary Science"})
    
    print(f"\n\n{result.deleted_count} documents deleted\n")
    print("After delete:")
    docs = db_col_student.find({}, {"_id": 0})
    for i in docs:
        print(i)

Connecting to MongoDB...
Before delete:
{'name': 'Amy', 'MatrNum': '0000154', 'course': 'BSc Mechanical Engineering'}
{'name': 'Michael', 'MatrNum': '0000482', 'course': 'BSc Mechanical Engineering'}
{'name': 'Sandra', 'MatrNum': '0000962', 'course': 'BSc Veterinary Science'}
{'name': 'Ben', 'MatrNum': '0001058', 'course': 'BSc Systems Engineering'}
{'name': 'Tom', 'MatrNum': '0000845', 'course': 'BSc Mechanical Engineering'}
{'name': 'Harry', 'MatrNum': '0005436', 'course': 'BSc Life Sciences'}
{'name': 'Archie', 'MatrNum': '0000968', 'course': 'BSc Life Sciences'}
{'name': 'Philip', 'MatrNum': '0000824', 'course': 'BSc Nursing'}
{'name': 'Stacy', 'MatrNum': '0000156', 'course': 'BSc Veterinary Science'}
{'name': 'Betty', 'MatrNum': '0000578', 'course': 'BSc Mechanical Engineering'}
{'name': 'Richard', 'MatrNum': '0002548', 'course': 'BSc Veterinary Science'}


3 documents deleted

After delete:
{'name': 'Amy', 'MatrNum': '0000154', 'course': 'BSc Mechanical Engineering'}
{'name': 'Mi

## Aggregation Framework
We’ll perform a simple aggregation to count the number of occurrences for each course, and sort the result by `count` and `_id`.

In [25]:
with MongoDBClient() as mClient:
    pipeline = [ {"$group": {"_id": "$course", "count": {"$sum": 1}}},
                 {"$sort": {"count": -1}}]
    list = mClient.mydatabase.student.aggregate(pipeline)
    
    for t in list:
        print(t)

Connecting to MongoDB...
{'_id': 'BSc Mechanical Engineering', 'count': 4}
{'_id': 'BSc Life Sciences', 'count': 2}
{'_id': 'BSc Nursing', 'count': 1}
{'_id': 'BSc Systems Engineering', 'count': 1}
Exiting MongoDB...


## Delete a collection

In [26]:
with MongoDBClient() as mClient:
    # get the existing collection
    col_list = mClient.mydatabase.list_collection_names()
    print("Existing collections:")
    for coll in col_list:
        print(coll)
    
    mClient.mydatabase.drop_collection("student")
    
    col_list = mClient.mydatabase.list_collection_names()
    print("\n\nAfter dropping the collection:")
    for coll in col_list:
        print(coll)

Connecting to MongoDB...
Existing collections:
faculty
student


After dropping the collection:
faculty
Exiting MongoDB...


## Delete a database

In [27]:
with MongoDBClient() as mClient:
    # get any existing databases
    dblist = mClient.list_database_names()
    print("Existing databases:")
    for db in dblist:
        print(db)
        
    mClient.drop_database("mydatabase")
    
    dblist = mClient.list_database_names()
    print("\n\nAfter dropping the database:")
    for db in dblist:
        print(db)

Connecting to MongoDB...
Existing databases:
admin
config
local
mydatabase


After dropping the database:
admin
config
local
Exiting MongoDB...
